In [ ]:
import numpyro
import numpy as np
import pandas as pd
from jax.nn import softmax
import jax.numpy as jnp

import rt_from_frequency_dynamics as rf

In [ ]:
data_name = "variants-us"
raw_seq = pd.read_csv(f"../data/{data_name}/{data_name}_location-variant-sequence-counts.tsv", sep="\t")

In [ ]:
raw_seq

In [ ]:
# Locations to run
locations = pd.unique(raw_seq["location"])
#locations = ["Washington"]

In [ ]:
# Defining Lineage Models
tau = 5.2 # Generation time
LM_MLR = rf.MultinomialLogisticRegression(tau)

In [ ]:
# Params for fitting
opt = numpyro.optim.Adam(step_size=1e-2)
iters = 100_000
num_samples = 3000
save = False
load = False

In [ ]:
# Paths for export
path_base = f"../estimates/{data_name}"

# Running models and exporting results

In [ ]:
def get_location_MLRData(rs, loc):
    rs_l = rs[rs.location==loc].copy()
    return rf.MLRData(rs_l)

In [ ]:
def fit_SVI_locations(rs, locations, LM, opt, **fit_kwargs):
    n_locations = len(locations)
    MP = rf.MultiPosterior()
    for i, loc in enumerate(locations):
        VD = get_location_MLRData(rs, loc)
        PH = rf.fit_SVI(VD, LM, opt, name=loc, **fit_kwargs)
        MP.add_posterior(PH)
        print(f'Location {loc} finished ({i+1}/{n_locations}).')
    return MP

In [ ]:
# Running GARW model
MP_MLR = fit_SVI_locations(raw_seq, locations, 
                             LM_MLR, opt, 
                             iters=iters, num_samples=num_samples, save=save, load=load)

# Making forecasts

In [ ]:
def posterior_frequency(dataset, X):
    """
    Use posterior beta to get posterior frequenicies based on matrix X.
    """
    beta = jnp.array(dataset.posterior["beta"])[1]
    logits = jnp.dot(X, beta) # Logit frequencies by variant
    return softmax(logits, axis=-1)

In [ ]:
PH = MP_MLR.get("Washington") # Unpack posterior for Washington
VD = PH.data
last_T = len(VD.dates) # Get end of fitting period
forecast_L = 14 # How far ahead to forecast

# Creating feature matrices
X = LM_MLR.make_ols_feature(start=0, stop=last_T) 
X_forecast = LM_MLR.make_ols_feature(start=last_T, stop=last_T + forecast_L)

# Computing posterior frequencies
freq = posterior_frequency(PH.dataset, X)
freq_forecast = posterior_frequency(PH.dataset, X_forecast)

# Making figures

In [ ]:
from rt_from_frequency_dynamics.plotfunctions import *
ps = DefaultAes.ps
alphas = DefaultAes.alphas

v_colors =["#2e5eaa", "#5adbff",  "#56e39f","#b4c5e4", "#f03a47",  "#f5bb00", "#9e4244","#9932CC", "#808080"] 
v_names = ['Alpha', 'Beta', 'Delta', 'Epsilon', 'Gamma', 'Iota', 'Mu', 'Omicron', 'other']
color_map = {v : c for c, v in zip(v_colors, v_names)}

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.transforms as mtransforms

font = {'family' : 'Helvetica',
        'weight' : 'light',
        'size'   : 32}

matplotlib.rc('font', **font)

In [ ]:
def plot_MLR_frequencies(ax, t, freq, ps, alphas, colors):
    
    # Compute quantiles
    V = []
    for p in ps:
        up = 1/2 + p/2
        lp = 1/2 - p/2
        Q = jnp.quantile(freq, jnp.array([lp,up]), axis=1)
        V.append(Q)
    med = jnp.median(freq, axis=1) # Compute median
    
    T, N_variant = med.shape
    # Make figure
    for variant in range(N_variant):
        for i in range(len(ps)):
            ax.fill_between(t, V[i][0, :, variant], V[i][1, :, variant],
                            color=colors[variant], alpha=alphas[i])
        ax.plot(t, med[:, variant],
                color=colors[variant])

In [ ]:
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(nrows=1, ncols=1)
ax = fig.add_subplot(gs[0,0])

colors = [color_map[v] for v in VD.seq_names] # Mapping colors to observed variants

# Plot predicted frequencies
plot_MLR_frequencies(ax, X[:,-1], freq, ps, alphas, colors)
plot_MLR_frequencies(ax, X_forecast[:,-1], freq_forecast, ps, alphas, colors)

rf.plot_observed_frequency(ax, VD, colors) # Plot observed frequencies

ax.axvline(x=len(VD.dates)-1, color='k', linestyle='--') # Adding forecast cut off
rf.add_dates_sep(ax, rf.expand_dates(VD.dates, forecast_L), sep=20) # Adding dates
ax.set_ylabel("Variant frequency") # Making ylabel


# Make legend
patches = [matplotlib.patches.Patch(color=c, label=l) for l, c in color_map.items()]
legend = fig.legend(patches, list(color_map.keys()), ncol=len(color_map.keys()), loc="lower center")  
legend.get_frame().set_linewidth(2.)
legend.get_frame().set_edgecolor("k")
fig.tight_layout()
fig.subplots_adjust(bottom = 0.2)

In [ ]:
def figure_growth_advantage(ga_df, ps, alphas, colors):
    fig = plt.figure(figsize=(28, 20))
  
    variants = pd.unique(ga_df.variant)
    locations = pd.unique(ga_df.location)
    location_map = {l: i for i, l in enumerate(locations)}

    
    # Sort level of confidence  
    _lw = [1.5, 2.5, 3.5]
    
    # Top panel
    ax1 = fig.add_subplot(2,1,2)
    ax1.axhline(y=1, lw=2,linestyle='dashed', color="k")

    for v, var in enumerate(variants):
        this_lineage = ga_df[ga_df.variant == var]
        location_num = this_lineage["location"].map(location_map)
        ax1.scatter(location_num, this_lineage.median_ga.values, 
                    color=colors[v],
                    edgecolors="k",
                    s = 45,
                    zorder = 3)
        
        # Plot error bars for each level of credibility
        for i, p in enumerate(ps):
            _p = int(p * 100)
            l_err = this_lineage.median_ga.values - this_lineage[f"ga_lower_{_p}"].values
            r_err = this_lineage[f"ga_upper_{_p}"].values - this_lineage.median_ga.values 
            ax1.errorbar(location_num, this_lineage.median_ga.values, 
                         yerr=[l_err, r_err], 
                          fmt = 'none',
                         color = colors[v], elinewidth = _lw[i])
        
    
    # Adding state labels
    ax1.set_xticks(np.arange(0, len(locations), 1))
    ax1.set_xticklabels([l.replace("_", " ") for l in locations],  rotation =90)
    
    # Adding axis label
    ax1.set_ylabel("Growth Advantage")
    
    # Right plot
    ax2 = fig.add_subplot(2,1,1)
    ax2.axhline(y=1, lw=2, linestyle='dashed', color="k")

    violin_data = [ga_df[ga_df.variant == v].median_ga.values for v in variants]
    parts = ax2.violinplot(violin_data, 
                           showmeans=False, 
                           showmedians=False, 
                           showextrema=False)
    
    for i, pc in enumerate(parts["bodies"]):
        pc.set_facecolor(colors[i])
        pc.set_edgecolor('black')
        pc.set_alpha(1)
    
    for v, var in enumerate(variants):
        this_lineage = ga_df[ga_df.variant == var]
        ax2.scatter([v+1 + np.random.normal(0, 0.02, 1) for i in range(len(this_lineage))],
                    this_lineage.median_ga.values, 
                    color=colors[v],
                    edgecolors="k",
                    s = 45,
                    zorder = 3) 
    
    ax2.set_ylabel("Median Growth Advantage")
    ax2.set_xticks(np.arange(1, len(variants)+1, 1))
    ax2.set_xticklabels(variants)
    
    axs = [ax2, ax1]
    labels = ["(a)", "(b)"]
    
    for label, ax in zip(labels, axs):
        trans = mtransforms.ScaledTranslation(-32/72, 8/72, fig.dpi_scale_trans)
        ax.text(0.0, 1.0, label, transform=ax.transAxes + trans,
            fontsize='large', va='bottom', fontfamily='serif')
        
    return fig

In [ ]:
ga_df_MLR = rf.gather_ga(MP_MLR, ps)

In [ ]:
ga_df_MLR.to_csv(f"{path_base}/{data_name}_ga-combined-MLR.tsv",  encoding='utf-8', sep='\t', index=False)

In [ ]:
ga_df_MLR = pd.read_csv(f"{path_base}/{data_name}_ga-combined-MLR.tsv", sep='\t',)

In [ ]:
colors = [color_map[v] for v in pd.unique(ga_df_MLR.variant)]
fig = figure_growth_advantage(ga_df_MLR, ps, alphas, colors)

In [ ]:
fig.savefig("../manuscript/figs/fig_MLR_growth_advantages_supp.png", facecolor="w", bbox_inches='tight')